## 관련 모듈 설치

In [ ]:
pip install azure-cognitiveservices-vision-customvision
pip install pymysql

## 관련 모듈 임포트

In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateBatch, ImageUrlCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid
import pymysql

## MariaDB 연결객체 생성

In [ ]:
db = pymysql.connect(host='localhost', port=3306, db='mysql', user='root', passwd='root', charset='utf8')
cur = db.cursor()

## Credential 설정

In [ ]:
TRAIN_ENDPOINT = "{ENDPOINT FOR TRAINING}"
training_key = "{APIKEY FOR TRAINING}"
PRE_ENDPOINT = "{ENDPOINT FOR PREDICTION}"
prediction_key = "{APIKEY FOR PREDICTION}"
prediction_resource_id = "{PREDICTION ID}"

In [ ]:
train_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAIN_ENDPOINT, train_credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PRE_ENDPOINT, prediction_credentials)

## 학습 모델 설정

In [ ]:
publish_iteration_name = "classifyModel"

## 프로젝트 생성

In [ ]:
project_name = uuid.uuid4()
project = trainer.create_project("minipjt_test")

## 태그 생성

In [ ]:
tag_lst = ['신라면', '진라면 매운맛', '올리브짜파게티', '안성탕면', '무파마탕면', '팔도비빔면', '열라면', '스낵면', '육개장라면', '팔도 참깨라면', 'CJ 비비고 왕교자', '비비고 곤드레나물밥', '잔슨빌소시지볶음밥', '하림 모짜렐라 치즈볼', '스모크 닭다리', '오뚜기 맛있는 오리지널핫도그', '목우촌모짜렐라크리스피핫도그', '교자만두 고기', '고향만두', '노엣지 꽉찬토핑피자 베이컨파이브치즈', '누드 누가바', '부라보바 화이트바닐라', '끌레도르 크림치즈바', '국화빵 인절미', '쿠키오', '설레임아라비카커피쉐이크', '투게더 녹차 초코', '녹차마루', '끌레도르 초코브라우니', '해태초코청크마루', '도라에몽 엔초', '가나소프트콘', '빵빠레 파스퇴르', '끌레도르 솔티드카라멜', '바나나바', '구구크러스터 화이트', '요맘때 로어슈거 딸기콘', '누가바초코', '설레임 바나나쉐이크', '빵빠레 그릭요거트', '동원 마일드참치', '스팸 마일드', '리챔 오리지널', '황도', '동원 꽁치', '스위트콘', '동원골뱅이', '비비고육개장', '비비고 설렁탕', '양반참치죽', '3분미트볼', '양반 단호박죽', '복음자리 딸기', '오랑지나', '이프로 부족할때 복숭아', '게토레이 레몬향', '바나나킥', '새우깡', '꼬북칩 달콤한 콩가루 인절미맛', '꼬북칩 콘스프맛', '카스타드', '수미칩 어니언', '알새우칩', '야채크래커', '양파링 오리지날', '오감자', '오레오', '오징어집', '오징어땅콩', '쵸코하임', '쿠크다스 화이트', '프링글스 오리지날', '다이제', '뻥이요', '계란과자', '엄마손파이', '버터와플', '에어 베이크드 팝칩', '아침에주스', '서울우유', '데자와로얄밀크티', '비타500', '갈아만든배', '코카콜라', '밀키스', '데미소다 복숭아', '핫식스', '쿨피스 복숭아', '칠성사이다', '미에로화이바', '웅진초록매실', '하루야채 퍼플', '포도봉봉', '매일우유', '따옴 사과주스', '트레비 레몬', '포카리스웨트', '쿠퍼스 헛개차', '바밤바', '뿌셔뿌셔 불고기맛']
for i in range(50):
    globals()['tag{}'.format(i)] = trainer.create_tag(project.id, tag_lst[i])

## 이미지 업로드

In [ ]:
image_lst = []
for i in range(50):
    sql = "select * from customvision where tag ="+str(i)
    cur.execute(sql)
    for tag, img_url in list(cur):
        image_lst.append(ImageUrlCreateEntry(url=img_url, tag_ids=[globals()['tag{}'.format(i)].id]))

In [ ]:
trainer.create_images_from_urls(project.id, ImageFileCreateBatch(images=image_lst))

## 학습

In [ ]:
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)